In [1]:
import pandas as pd
import numpy as np
from dimod import BinaryQuadraticModel
from dwave.system import LeapHybridSampler

import warnings
warnings.filterwarnings('ignore')

In [2]:
df1 = pd.read_csv('foot.csv')
df2 = pd.read_csv('foot2.csv')

df1['Player'] = df1['Player'].str.replace(r"\(.*\)","")
df2['Player'] = df2['Player'].str.replace(r"*","")
df2['Player'] = df2['Player'].str.replace(r"+","")
df1['dollar_value'] = df1['dollar_value'].str.replace(r"$","")

df1['Player'] = df1['Player'].str.replace(" ","")
df2['Player'] = df2['Player'].str.replace(" ","")

df1.dollar_value = df1.dollar_value.astype(int)

In [3]:
df = df1.merge(df2, on ='Player')

df = df.dropna()

In [4]:
lst_qb = list(df[df.FantPos == 'QB'].index)

In [5]:
values = list(df['FantPt'])
weights = list(df['dollar_value'])
volumes = [x/x for x in range(1, len(values))]

n = len(values)
variables = list(range(n))

In [6]:
bqm = BinaryQuadraticModel('BINARY')

variables = [bqm.add_variable(v, -values[v]) for v in variables]

slacks_volume = bqm.add_linear_equality_constraint(
[(x, v) for x, v in zip(variables, volumes)],
constant=-15,
lagrange_multiplier=400
)

slacks_volume = bqm.add_linear_equality_constraint(
[(x, 1) for x in variables if x in lst_qb],
constant=-1,
lagrange_multiplier=100
)


slacks_weight = bqm.add_linear_inequality_constraint(
[(x, v) for x, v in zip(variables, weights)],
constant=-75,
lagrange_multiplier=600,
label = 'weight'
)

In [7]:
sampler = LeapHybridSampler(token = "KmJQ-eb7dea9880650063660800305a6750d9fe70bb21")
response = sampler.sample(
    bqm, time_limit=25,
    )
best_solution = response.first.sample

indices = []

for i, v in best_solution.items():
    if not str(i).startswith('s'):
        if v != 0:
            indices.append(i)

In [8]:
print(df.iloc[indices][['Player','Pick', 'FantPos', 'dollar_value' ]])

             Player  Pick FantPos  dollar_value
5      DerrickHenry     7      RB            18
17        NickChubb    25      RB            11
31  DavidMontgomery    41      RB             7
33        MikeEvans    43      WR             7
35      AdamThielen    46      WR             7
38     AaronRodgers    49      QB             6
39       JoshJacobs    51      RB             6
48    RyanTannehill    68      QB             3
49     DavidJohnson    69      RB             3
50    JamesRobinson    70      RB             3
52    ChaseClaypool    81      WR             2
57      KenyanDrake    92      RB             1
60     CurtisSamuel   121      WR             0
61      NyheimHines   128      RB             0
62    RonaldJonesII   131      RB             0
81         DrewLock   367      QB             0
